In [1]:
import pylab as plt
import pandas as pd
import numpy as np
import sklearn.impute as skin
from sqlalchemy import create_engine
import locale
import os
locale.setlocale(locale.LC_ALL,'fr_FR')
output_folder=r"C:\Users\berend\Documents\Python_Scripts\vrije_stage\Results_uncertainty"
hour=6
from all_own_functions import cnfl,value_filtering

C:\Users\berend\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
from All_standard_variables import conv_dict, dtype_dict
# Used columns from database
lab_cols=['pat_bd','pat_datetime','lab_bl_b2m', 'lab_bl_bil_d', 'lab_bl_bil_i', 'lab_bl_ca2', 'lab_bl_catot', 'lab_bl_cc', 'lab_bl_cl', 'lab_bl_cr', 'lab_bl_CRP', 'lab_bl_f', 'lab_bl_gluc', 'lab_bl_hb', 'lab_bl_ht', 'lab_bl_k', 'lab_bl_lactate', 'lab_bl_leuco', 'lab_bl_mg', 'lab_bl_na', 'lab_bl_tr', 'lab_bl_ur', 'pat_hosp_id', 'Status']
# datetime columns
pars_dates=['pat_bd','pat_datetime']
# read in data as dataframe (kind of table with an index)
df=pd.read_csv(f"total_database_{hour}hour.csv",delimiter=';',usecols=lab_cols,header=0,converters=conv_dict,parse_dates=pars_dates)
print('Done')

C:\Users\berend\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
Done


In [3]:
import datetime as dt
if 'pat_bd' in list(df.columns):
    df.replace(to_replace='NULL', value=np.nan,inplace=True)            # Replace NULL with NaN
    df=value_filtering(df)                                              # Delete all rows with no values
    df['pat_datetime_temp']=pd.to_datetime(df['pat_datetime']).dt.date  # set datetime object to days object
    df['pat_bd']=pd.to_datetime(df['pat_bd']).dt.date                   # Same as above
    df['Age'] = (df['pat_datetime_temp'] - df['pat_bd']).dt.days        # Calculate age
    df['Age']=df['Age'].divide(365)                                     # Calculate age in years
    df=df.drop('pat_datetime_temp',axis=1)                              # Remove obsolete columns
    df=df.drop('pat_datetime',axis=1)
    df=df.drop('pat_bd',axis=1)
df['pat_hosp_id'] = df['pat_hosp_id'].astype('int')                     # Set patientnumber as integer instead of float

In [4]:
import typing
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler,RobustScaler

class Wraptastic:
    """
    Class of functions to apply sklearn transformers (i.e. RobustScaler) while remaininig the dataframe structure of the variable instead of an array
    """
    def __init__(self, transform: typing.Callable):
        self.transform = transform

    def __call__(self, df):
        transformed = self.transform.fit_transform(df.values)
        return pd.DataFrame(data=transformed, columns=df.columns, index=df.index)

# Categorise Age
if any(df['Age'] > 12) == True:
    df.loc[(df['Age'] > 12),'Age']=50
    df.loc[(df['Age'] <= 12) & (df['Age'] > 4),'Age']=51
    df.loc[(df['Age'] > 1) & (df['Age'] <=4),'Age']=52
    df.loc[(df['Age'] > 0.5)  & (df['Age'] <=1),'Age']=53
    df.loc[(df['Age'] <= 0.5),'Age']=54
    df['Age'] = df['Age'].subtract(50)
    df['Age'] = df['Age'].astype('int32')

# Select numerical columns for scaling
float_columns=list(df.select_dtypes(include=['float64','float32','int32']).columns)

ss=Wraptastic(RobustScaler())
float_columns.remove('pat_hosp_id') # No use of scaling patient numbers

df_temp=df[float_columns]
df_temp=(df_temp.groupby('Age',sort=False).apply(ss).drop('Age',axis="columns")) # apply scaling to the Age categories
float_columns.remove('Age') # No use of scaling the age categories
df[float_columns]=df_temp
print(df.info())

C:\Users\berend\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:993: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
C:\Users\berend\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1370: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input, interpolation=interpolation
C:\Users\berend\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:993: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
C:\Users\berend\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1370: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input, interpolation=interpolation
C:\Users\berend\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:993: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
C:\Users\berend\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1370

In [5]:
from statsmodels.stats.descriptivestats import sign_test
from tsfresh.feature_extraction.feature_calculators import abs_energy,number_cwt_peaks
from all_own_functions import fft_feat

def lab_values_feature_building(dfl,columns,pat):
    """"
    Function which calculates the specific features per variable per patient in the dataframe
    Function selects variables based on type of feature (i.e. float/object)
     """
    group=dfl
    df=pd.DataFrame()
    if 'work' in columns:
        group['work']=WOB_calc(group)
        group['hr_rr_ratio']=hr_rr_ratio(group)
    for colum in columns:
        if (group[colum].dtypes == float):
            temp=pd.DataFrame(group[colum].describe().to_numpy()[None], # Calculating the descriptive statistics for numerical variables
            columns=[colum+'_count',colum+'_mean',colum+'_std',colum+'_min',colum+'_25%',colum+'_50%',colum+ '_75%',colum+'_max'],index=[pat]) # Naming columns
            l=[colum+'_count',colum+'_std',colum+'_max']
            temp=temp[l]
            #skew=pd.Series(group[colum].skew(),name=(colum+'_skew'))
            #kurt=pd.Series(group[colum].kurtosis(),name=(colum+'_kurtosis'))
            #high_mean=pd.Series(sign_test(group[colum],mu0=group[colum].mean()),name=(colum+'_count_above_mean'))       # counts above mean
            #high_median=pd.Series(sign_test(group[colum],mu0=group[colum].median()),name=(colum+'_count_above_median')) # counts above median
            #abs_energy=pd.Series(abs_energy(group[colum]),name=(colum+'_abs_energy'))                                   # Absolute energy of the signal
            #cwt_peaks=pd.Series(abs_energy(group[colum]),name=(colum+'_number_CWT_peaks'))                              # Amount of peaks in the continuous wavelet analysis
            #fft_aggr=fft_feat(group[colum],pat,colum)                                                                   # Descriptive statistics in the frequency domain
            df=pd.concat([temp,df],axis=1)
            #df.append(skew)
            #df.append(kurt)
            #df.append(high_mean)
            #df.append(high_median)
            #df.append(cwt_peaks)
            #df.append(fft_aggr)
        elif (group[colum].dtypes == object):

            temp=pd.DataFrame(group[colum].describe().to_numpy()[None],  #calculating the descriptive statistics for categorical variables
            columns=[colum+'_count',colum+'_unique',colum+'_top',colum+'_freq'],index=[pat])
            
            df=pd.concat([temp,df],axis=1)
        else:
            temp=pd.DataFrame(group[colum].describe(datetime_is_numeric=True)['max'], # Calculating the descriptive statistics for the other variables
            columns=[colum+'_max'],index=[pat])  #[colum+'_count',colum+'_mean',colum+'_min',colum+'_25%',colum+'_50%',colum+'_75%',colum+ '_max'],index=[pat])
            df=pd.concat([temp,df],axis=1)
        del temp        
    return df     
    
def population_descriptives(df,columns):
    """
    Calculate the features based from the whole population/ dataframe
    Takes in the columns from which the features need to be calculated
    Also adds a for currrent status
    """
    grouped = df.groupby('pat_hosp_id',sort=False)                              # group dataframe per patient
    df_feat=grouped.get_group((list(grouped.groups)[0]))                        # get first patient
    df_feat=lab_values_feature_building(df_feat,columns,df_feat['pat_hosp_id'].iloc[0]) # calculate features for one patient (needed for intial dataframe for append function)
    for pat,group in grouped:        
        df_temp = lab_values_feature_building(group,columns,pat)                # loop over all patients
        if 'Death' in group['Status'].unique():                                 #add label for deceased patients
            df_temp['Label']='Death'
        else:
            df_temp['Label']='Alive'
        df_feat=df_feat.append(df_temp)                                         # append dataframe with total dataframe
        del df_temp
    return df_feat  

In [6]:
column_list=(list(df.columns))                                        # specifying which columns need to be taken into account
column_list.remove('Status')    
column_list.remove('pat_hosp_id')
df_try_full=population_descriptives(df,column_list)                   # using function described above
df_try_full = df_try_full[~df_try_full.index.duplicated(keep='last')] # eliminate double patients (first patient is calculated double)
print(df_try_full.info())                                             # show the dataframe
print(df_try_full.head())
print(df_try_full['Label'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6743 entries, 1936 to 9996691
Data columns (total 62 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age_max               6743 non-null   float64
 1   lab_bl_ur_count       6743 non-null   float64
 2   lab_bl_ur_std         453 non-null    float64
 3   lab_bl_ur_max         1301 non-null   float64
 4   lab_bl_tr_count       6743 non-null   float64
 5   lab_bl_tr_std         1094 non-null   float64
 6   lab_bl_tr_max         2853 non-null   float64
 7   lab_bl_na_count       6743 non-null   float64
 8   lab_bl_na_std         3792 non-null   float64
 9   lab_bl_na_max         5228 non-null   float64
 10  lab_bl_mg_count       6743 non-null   float64
 11  lab_bl_mg_std         494 non-null    float64
 12  lab_bl_mg_max         1530 non-null   float64
 13  lab_bl_leuco_count    6743 non-null   float64
 14  lab_bl_leuco_std      658 non-null    float64
 15  lab_bl_leuco_ma

In [ ]:
"""
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
pdf=PdfPages('Scatterplots_lab.pdf')
for column in list(df.columns):
    try:
        sns.scatterplot(data=df_try_full,x=column+'_max',y=column+"_count",hue='Label',style='Age_max')
        plt.title(f'Scatterplot of count and max value of {column}')
        plt.legend(loc=1,fontsize='xx-small')
        fig=plt.gcf()
        pdf.savefig(fig)
        plt.close()
        sns.scatterplot(data=df_try_full,x=column+'_std',y=column+"_count",hue='Label',style='Age_max')
        plt.title(f'Scatterplot of count and min value of {column}')
        plt.legend(loc=1,fontsize='xx-small')
        fig=plt.gcf()
        pdf.savefig(fig)
        plt.close()
    except ValueError:
        print(column)
pdf.close()
"""

In [8]:
# specify datatypes for loading data
# cnfl is function for converting strings to floats
conv_pice={'BaseExcess':cnfl,'FiO2':cnfl,'PaO2':cnfl,'PIM2Score':cnfl,'PIM2Mort':cnfl,'PIM3Score':cnfl,
'PIM3Mort':cnfl,'GlucoseMax':cnfl,'PaO2Min':cnfl,'PCO2Max':cnfl,'PHMax':cnfl,'PHMin':cnfl,'PotassiumMax':cnfl,'PTMax':cnfl,'TemperatureMax':cnfl,'TemperatureMin':cnfl,
'BicarbonateMin':cnfl,'BicarbonateMax':cnfl,'UreaMax':cnfl,'WhiteBloodCountMin':cnfl,'PRISM4Mortality':cnfl,'Age(days)':cnfl}

df_pice=pd.read_csv('scores.csv',converters=conv_pice,header=0,delimiter=';') # load in data

df_pice.drop(df_pice.tail(2).index,inplace=True)                              # drop last column which is not a patient
df_pice.dropna(axis=0,how='all',inplace=True)                                 # drop all rows with only NaN
#del df_pice['AdmissionDate']                                                  # Admission data is deemed not relevant

index_names = df_pice[df_pice['HospitalNumber'] == 'RPH'].index               # remove column with RPH as patientnumber
df_pice.drop(index_names, inplace=True)

# identifying duplicate patients                                    
a=df_pice[df_pice['HospitalNumber'].duplicated(keep=False)]                   
a=a[a['HospitalNumber'].duplicated(keep='first')]
duplicate_patients=a['HospitalNumber'].tolist()

# set patientnumber as index
df_pice=df_pice.set_index('HospitalNumber')                                  
df_pice=df_pice.sort_index()


# Categorize the RiskDiagnoses
pimLowRisk = ["Asthma","Bronchiolitis","Croup","ObstructiveSleepApnea","DiabeticKetoacidosis",'SeizureDisorder']
pimHighRisk = ["CerebralHemorrhage","CardiomyopathyOrMyocarditis","HIVPositive","HypoplasticLeftHeartSyndrome","NeurodegenerativeDisorder","NecrotizingEnterocolitis"]
pimVeryHighRisk = ["CardiacArrestInHospital","CardiacArrestPreHospital","SevereCombinedImmuneDeficiency","LeukemiaorLymphoma","BoneMarrowTransplant","LiverFailure"]

# if Riskdiagnoses not known fill in unknown
df_pice['RiskDiagnoses'] = (df_pice['RiskDiagnoses'].fillna(value='Unknown'))

for i,row in df_pice.iterrows():
    try:
        if any(substring in df_pice.loc[i,'RiskDiagnoses'] for substring in pimVeryHighRisk) == True:
            df_pice.loc[i,'RiskDiagnoses'] = 3
        elif any(substring in df_pice.loc[i,'RiskDiagnoses'] for substring in pimHighRisk)==True :
            df_pice.loc[i,'RiskDiagnoses']= 2
        elif any(substring in df_pice.loc[i,'RiskDiagnoses'] for substring in pimLowRisk)==True:
            df_pice.loc[i,'RiskDiagnoses'] = 1
        else: df_pice.loc[i,'RiskDiagnoses'] = 0
    except TypeError:
        continue

# Remove status from df_pice, it is already included in the other dataframe
if 'Status' in list(df_pice.columns):
    del df_pice['Status']
print(df_pice.info())

<class 'pandas.core.frame.DataFrame'>
Float64Index: 13791 entries, 219.0 to 526480713.0
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   AdmissionDate             13791 non-null  object 
 1   Age(days)                 13791 non-null  float64
 2   RiskDiagnoses             13791 non-null  object 
 3   Urgency                   13791 non-null  object 
 4   Recovery                  13791 non-null  object 
 5   Ventilated                13791 non-null  object 
 6   AdmissionPupils           13187 non-null  float64
 7   SystolicBP                11269 non-null  float64
 8   BaseExcess                6433 non-null   float64
 9   FiO2                      6815 non-null   float64
 10  PaO2                      5739 non-null   float64
 11  PIM2Score                 13791 non-null  float64
 12  PIM2Mort                  13791 non-null  float64
 13  PIM3Score                 13791 non-null  float64

In [9]:
# remove duplicate patients
#df_pice=df_pice[~df_pice.index.duplicated(keep='last')]
print(df_pice.head())
# Applying robust scaling to numerical columns of PICE
scaler=RobustScaler()
float_columns_pice = list(df_pice.select_dtypes(include=['float64']).columns)
df_pice_temp=df_pice[float_columns_pice]
df_pice_scaled=pd.DataFrame(scaler.fit_transform(df_pice_temp), columns=float_columns_pice,index=df_pice_temp.index)
print(df_pice_scaled.head())
df_pice[float_columns_pice]=df_pice_scaled

                   AdmissionDate    Age(days) RiskDiagnoses      Urgency  \
HospitalNumber                                                             
219.0             3-3-2019 18:28  5186.769531             1  NotElective   
377.0            21-8-2003 03:00   346.125000             0  NotElective   
792.0            15-9-2019 16:33     4.689583             2  NotElective   
1000.0          20-11-2003 15:00  1219.625000             0     Elective   
1469.0          23-11-2007 13:00  4091.541748             0     Elective   

                               Recovery Ventilated  AdmissionPupils  \
HospitalNumber                                                        
219.0                        NoRecovery      False              0.0   
377.0                        NoRecovery      False              0.0   
792.0                 PostCardiacByPass       True              0.0   
1000.0                       NoRecovery       True              0.0   
1469.0          PostNonCardiacProcedure  

In [10]:
from collections import Counter
# Sorting PICE in two dataframes
# one with only Scores and the other with only modalities
all_pice_columns=list(df_pice.columns)
score_mor=['PIM3Score', 'PIM3Mort', 'PIM2Mort', 'PRISM4Mortality','PIM2Score','PRISM3Neuro','PRISM3Score']
pice_items= list((Counter(all_pice_columns)-Counter(score_mor)).elements())
df_pice_scores=df_pice[score_mor]
df_pice_modalities=df_pice[pice_items]
print(df_try_full.info())
print(pice_items)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6743 entries, 1936 to 9996691
Data columns (total 62 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age_max               6743 non-null   float64
 1   lab_bl_ur_count       6743 non-null   float64
 2   lab_bl_ur_std         453 non-null    float64
 3   lab_bl_ur_max         1301 non-null   float64
 4   lab_bl_tr_count       6743 non-null   float64
 5   lab_bl_tr_std         1094 non-null   float64
 6   lab_bl_tr_max         2853 non-null   float64
 7   lab_bl_na_count       6743 non-null   float64
 8   lab_bl_na_std         3792 non-null   float64
 9   lab_bl_na_max         5228 non-null   float64
 10  lab_bl_mg_count       6743 non-null   float64
 11  lab_bl_mg_std         494 non-null    float64
 12  lab_bl_mg_max         1530 non-null   float64
 13  lab_bl_leuco_count    6743 non-null   float64
 14  lab_bl_leuco_std      658 non-null    float64
 15  lab_bl_leuco_ma

In [11]:
from sklearn.preprocessing import LabelBinarizer
from collections import Counter 
# copy df as failsafe
df_coms=df_try_full
# binarize the status of a patient Alive = 0, Death = 1
lb=LabelBinarizer()
df_coms['Label']=lb.fit_transform(df_coms['Label'])

# Selecting the correct datatype for all columns
a=list(df_coms.columns)
a.remove('Label')
for column in a:
    if 'top' in column:
        df_coms[column]=df_coms[column].astype('object')
    elif 'count' in column:
        df_coms[column]=df_coms[column].astype('int8')
    else:
        df_coms[column]=df_coms[column].astype('float64')

# Join PICE and PICURED
df_com=df_coms.join(df_pice,how='left')
df_com = df_com.reset_index()

# Add variable for rehospitalisation
for i,row in df_com.iterrows():
    if  df_com.index[i] in duplicate_patients:
        df_com.loc[i,'ReHospitalisation'] = 1
    else:
        df_com.loc[i,'ReHospitalisation'] = 0

# Seperate data from  label
x=df_com.drop(['index','Label'],axis=1)
y=df_com['Label']

print(x.info())

print(x.head())
print(y.value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10156 entries, 0 to 10155
Columns: 104 entries, Age_max to ReHospitalisation
dtypes: float64(75), int8(20), object(9)
memory usage: 6.7+ MB
None
   Age_max  lab_bl_ur_count  lab_bl_ur_std  lab_bl_ur_max  lab_bl_tr_count  \
0      0.0                0            NaN            NaN                0   
1      0.0                1            NaN      -0.111111                1   
2      1.0                0            NaN            NaN                1   
3      2.0                0            NaN            NaN                0   
4      2.0                0            NaN            NaN                0   

   lab_bl_tr_std  lab_bl_tr_max  lab_bl_na_count  lab_bl_na_std  \
0            NaN            NaN                1            NaN   
1            NaN      -0.904762                4       0.125000   
2            NaN      -1.332258                3      15.992915   
3            NaN            NaN                6      19.170290   
4

In [ ]:
# Caculate correlation of data with label
c=x.corrwith(y,method='pearson')
d=x.corrwith(y,method='spearman')

# Select relevant features based on cut-off values
relevant_features_pe = c[c>0.05]
relevant_features_sp = d[d>0.05]

# Extract correlating features from main dataframe
x_pe=x[relevant_features_pe.index]
x_sp=x[relevant_features_sp.index]


# Top features from trained classifiers (linear svm, decisiontree, logistic regression)
#top=['lab_bl_catot_mean', 'lab_bl_catot_max', 'CPR24HourBefore', 'lab_bl_f_mean', 'lab_bl_na_count', 'lab_bl_lactate_mean', 'lab_bl_catot_std', 'lab_bl_f_min', 'lab_bl_k_count', 'lab_bl_bil_d_std', 'lab_bl_gluc_mean', 'lab_bl_gluc_max', 'lab_bl_ur_mean', 'lab_bl_ht_std', 'lab_bl_bil_i_std', 'lab_bl_na_std', 'Urgency', 'lab_bl_ur_max', 'LowRiskPrimary', 'lab_bl_catot_min', 'lab_bl_ht_max', 'lab_bl_leuco_std', 'lab_bl_cl_count', 'Cancer', 'lab_bl_CRP_std', 'lab_bl_k_mean', 'lab_bl_lactate_max']
#top=['lab_bl_na_count', 'lab_bl_catot_mean', 'lab_bl_k_count', 'lab_bl_hb_max', 'lab_bl_catot_max', 'PHMax', 'lab_bl_bil_d_std', 'lab_bl_na_std', 'PHMin', 'lab_bl_mg_std', 'lab_bl_ht_mean', 'Recovery', 'lab_bl_cl_count', 'lab_bl_mg_max', 'lab_bl_hb_mean', 'CPR24HourBefore', 'lab_bl_catot_std', 'lab_bl_k_std']

# Extract top features from dataframe
#x_top=x[top]
#x_top = x_top.loc[:,~x_top.columns.duplicated()]
#print(x_top.info())


In [ ]:
from sklearn.preprocessing import OneHotEncoder,PolynomialFeatures,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB,CategoricalNB,BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF,DotProduct,Matern,RationalQuadratic,WhiteKernel
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis,LinearDiscriminantAnalysis
from sklearn.neighbors import NearestCentroid,NeighborhoodComponentsAnalysis,KNeighborsClassifier
from sklearn.linear_model import LinearRegression,MultiTaskLasso,ElasticNetCV,BayesianRidge,LogisticRegression,RidgeClassifierCV,LassoCV,LassoLarsCV,OrthogonalMatchingPursuitCV
from sklearn.ensemble import AdaBoostRegressor,BaggingClassifier,RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score,StratifiedKFold,RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel,RFE,VarianceThreshold,RFECV,SelectPercentile, chi2,SelectKBest
from sklearn.svm import SVC,SVR,LinearSVC
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,plot_precision_recall_curve
from sklearn.metrics import roc_curve,roc_auc_score,plot_confusion_matrix,f1_score
from xgboost import XGBClassifier
from probatus.feature_elimination import ShapRFECV
import lightgbm
import xgboost
from all_own_functions import f_importances
import os
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

def machine_learning_function(x_train,x_test,y_train,y_test,model,wrapper=0):
    """
    Function which takes train and test data and uses it to train a specified model, also the option to use wrapper based feature selection model
    Also preforms dataimputation and onehotencoding
    """
    # specify types of variables, since it changes the imputation
    float_columns=list(x_train.select_dtypes(include=['float64']).columns)
    int_columns=list(x_train.select_dtypes(include=['int32']).columns)
    cat_list=list(x_train.select_dtypes(include=['object']).columns)
    
    # remove label if still in x_train
    if 'Label' in float_columns: float_colums.remove('Label')

    #imputation and one hot encoding
    float_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent'))])
    int_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='constant',fill_value=0))])
    cat_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='constant',fill_value='Unknown')),('onehot', OneHotEncoder(handle_unknown='ignore'))])

    # group all preprocessing
    preprocess = ColumnTransformer(transformers=[('float',float_transformer,float_columns),('int',int_transformer,int_columns),('cat',cat_transformer,cat_list)],                       remainder='passthrough')
    
    # create machine learning pipeline, wrapper specifies the feature selection
    if wrapper == 1:
        clf = Pipeline(steps=[('preprocessor',preprocess),('Feature_selection',SelectFromModel(LinearSVC(max_iter=3000))),
        ('classifier',model)])
    elif wrapper == 2:
        clf = Pipeline(steps=[('preprocessor',preprocess),('Feature_selection',SelectFromModel(DecisionTreeClassifier())),
        ('classifier',model)])
    elif wrapper == 3:
        clf = Pipeline(steps=[('preprocessor',preprocess),('Feature_selection',SelectFromModel(LogisticRegression())),
        ('classifier',model)])
    else:
        clf = Pipeline(steps=[('preprocessor',preprocess),('classifier',model)])

    # fit the pipeline to the data
    clf=clf.fit(x_train,y_train)
    # Predict the labels
    y_pred_clas=clf.predict(x_test)

    # Predict the probabilities, function depends on used classifier

    try:
        y_pred_prob=clf.decision_function(x_test)
    except:
        try:
            y_pred_prob=clf.predict_proba(x_test)
            y_pred_prob=y_pred_prob[:,1]
        except:
            y_pred_prob=y_pred_clas

    # failsafe to inpute NaN probabilities with 0
    inds = np.where(np.isnan(y_pred_prob))
    if inds:
        y_pred_prob=np.nan_to_num(y_pred_prob, nan=0)
    return clf, y_pred_clas,y_pred_prob, y_test,x_test

In [ ]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve,roc_auc_score,brier_score_loss
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
# speciify colours for plot later on
colours=['b-', 'g--', 'r:', 'c-.', 'm-+', 'y-*', 'k-o']


# Stratiefied fold for cros-validation
fold=StratifiedKFold(3)

# Names and classiefiers to be used in the loop
names = [ "Random Forest", "Gaus Naive Bayes",
        "Decision Tree Clas", "Sigmoid SVM", "QDA"]

classifiers = [
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1,class_weight={0:.1, 1:.9}),
    GaussianNB(priors=[0.9 , 0.1]),
    DecisionTreeClassifier(class_weight= {0:.1, 1:.9}),
    SVC(kernel='sigmoid',class_weight= {0:.1, 1:.9},probability=True),
    QuadraticDiscriminantAnalysis(priors=[0.9,0.1])
    ]

# Names and dataframes used in the loop
data_names=['All_Data','Pearson_Cor', 'Spearman_cor',"Wrap_Lin_SVM","Wrap_Dec_Tree","Wrap_Log_Reg"]
x_data=[x, x_pe, x_sp, x, x, x]

# split data in train and test data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=1 / 5, random_state=1)

# calculate ROC and AUC outside of loop
fpr_pim, tpr_pim, _ = roc_curve(y_test,  x_test['PIM3Score'])
auc_pim = roc_auc_score(y_test, x_test['PIM3Score'])

# check if result files do not already exist, updating pdf is not possible
try:
    pdf = PdfPages(os.path.join(output_folder,f"Figures of {hour} hour results.pdf"))
    pdf_ROC = PdfPages(os.path.join(output_folder,f"ROC of {hour} hour results.pdf"))
    pdf_cal = PdfPages(os.path.join(output_folder,f"Cal of {hour} hour results.pdf"))
    pdf_hist = PdfPages(os.path.join(output_folder,f"Hist of {hour} hour results.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"Figures of {hour} hour results.pdf"))
    os.remove(os.path.join(output_folder,f"ROC of {hour} hour results.pdf"))
    os.remove(os.path.join(output_folder,f"Results_{hour}hours_scores.txt"))
    os.remove(os.path.join(output_folder,f"Cal of {hour} hour results.pdf"))
    os.remove(os.path.join(output_folder,f"Hist of {hour} hour results.pdf"))
    pdf = PdfPages(os.path.join(output_folder,f"Figures of {hour} hour results.pdf"))
    pdf_ROC= PdfPages(os.path.join(output_folder,f"ROC of {hour} hour results.pdf"))
    pdf_cal = PdfPages(os.path.join(output_folder,f"Cal of {hour} hour results.pdf"))
    pdf_hist = PdfPages(os.path.join(output_folder,f"Hist {hour}  hour results.pdf"))

# loop over different data and feature selection techniques
for data_name, xd in zip(data_names, x_data):
    # transform columns based on dataframe used
    columns = xd.columns.tolist()
    x_train_d = x_train[columns]
    x_test_d = x_test[columns]
    
    # Add variable for wrapper based feature selection
    if 'Wrap' in data_name:
        wrapper += 1
    else:
        wrapper=0
    
    # create variables for temp storage lateron
    temp_fpr=dict()
    temp_tpr=dict()
    temp_auc=dict()
    temp_probtrue={}
    temp_probpred={}
    temp_score={}

    # loop over different classifiers
    for name, clf_s in zip(names, classifiers):

        clf,y_pred_clas,y_pred_prob,y_test,x_test_d = machine_learning_function(x_train_d,x_test_d,y_train,y_test,clf_s,wrapper)

        # calculate scoring metrics
        report=classification_report(y_test,y_pred_clas,target_names=['Alive','Death'])
        score=clf.score(x_test_d,y_test)
        average_precision = average_precision_score(y_test, y_pred_prob)
        f1_s=f1_score(y_test, y_pred_clas)

        # write scoring metrics to file
        with open(os.path.join(output_folder,f"Results_{hour}hours_scores.txt"),'a') as file:
            file.write(f"{data_name} with {name} Results for {hour} hours \n\n")
            file.write(f"Classification report \n {report} \n")
            file.write(f"Hold_out_scores {score} \n")
            file.write(f"Average precision score {average_precision} \n")
            file.write(f"F1 score {f1_s} \n\n\n")
        
        # plot confusion matrix
        plot_confusion_matrix(clf,x_test_d,y_test)
        plt.title(f"{data_name} with {name} classifier, Results from {hour} hour data")
        fig=plt.gcf()
        pdf.savefig(fig)
        plt.close(fig)

        # plot ROC with AUC
        fpr, tpr, _ = roc_curve(y_test,  y_pred_prob)
        auc = roc_auc_score(y_test, y_pred_prob)
        plt.plot(fpr,tpr,label=f"{name}, auc={auc}")
        plt.plot(fpr_pim,tpr_pim,label=f"PIM3Score, auc={auc_pim}")
        plt.title(f"{data_name} with {name} classifier, Results from {hour} hour data")
        plt.legend(loc=4)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        fig=plt.gcf()
        pdf.savefig(fig)
        plt.close(fig)

        # temporarly store ROC and AUC per classifier
        temp_fpr.update({f'{name}': fpr})
        temp_tpr.update({f'{name}': tpr})
        temp_auc.update({f'{name}': auc})

         # plot callibration plot with brier_loss score
        
        prob_true, prob_pred = calibration_curve(y_test,  y_pred_prob,normalize=True)
        plt.plot(prob_pred,prob_true,label=f"{name} ")
        plt.title(f"{data_name} with {name} classifier, Results from {hour} hour data")
        plt.legend(loc=4)
        plt.ylabel('Fraction of Positives')
        fig=plt.gcf()
        pdf.savefig(fig)
        plt.close(fig)

        # temporarly store ROC and AUC per classifier
        temp_probtrue.update({f'{name}': prob_true})
        temp_probpred.update({f'{name}': prob_pred})
        #temp_score.update({f'{name}':clf_score})
        
    #PLot every roc from the used classifier per feature selection method 
    a=0
    for k,v in temp_fpr.items():
        plt.plot(v,temp_tpr.get(k),colours[a],label=f"{k}, auc={temp_auc.get(k)}",linewidth=1.5,markersize=1)
        a= a+1
    plt.plot(fpr_pim,tpr_pim,'b->',label=f"PIM3Score, auc={auc_pim}",linewidth=1.5,markersize=1)
    plt.legend(loc=4,fontsize='xx-small')
    plt.title(f'{data_name} ROC from {hour} hour data')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    axes = plt.gca()
    axes.set_xlim([0,1])
    axes.set_ylim([0,1])
    fig=plt.gcf()
    pdf_ROC.savefig(fig)
    plt.close(fig)
    count=0

    #PLot calibration from the used classifier per feature selection method
    for k,v in temp_probtrue.items():
        plt.plot(temp_probpred.get(k),v,colours[count],label=f"{k}",linewidth=1.5,markersize=1)
        count += 1
    plt.legend(loc=4,fontsize='xx-small')
    plt.title(f'{data_name} Calibration Curve from {hour} hour data')
    plt.xlabel('Fraction of positives')
    axes = plt.gca()
    axes.set_xlim([0,1])
    axes.set_ylim([0,1])
    fig=plt.gcf()
    pdf_cal.savefig(fig)
    plt.close(fig)
    count=0

    # plot histogram from the used classifier per feature selection method
    for k,v in temp_probtrue.items():
        plt.hist(temp_probpred.get(k),range=(0,1),label=f"{k}",histtype='step',lw=2)
        count += 1
    plt.legend(loc=4,fontsize='xx-small')
    plt.title(f'{data_name} Calibration Curve from {hour} hour data')
    plt.xlabel('Fraction of positives')
    axes = plt.gca()
    axes.set_xlim([0,1])
    axes.set_ylim([0,1])
    fig=plt.gcf()
    pdf_hist.savefig(fig)
    plt.close(fig)


    del temp_fpr,temp_tpr,temp_auc
pdf.close()
pdf_ROC.close()
pdf_cal.close()
pdf_hist.close()

In [ ]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve,roc_auc_score
from matplotlib.backends.backend_pdf import PdfPages
# Same functions as before except now specified for Regression algorithms

names = ["Logistic Regression"
        ]

classifiers = [
    LogisticRegression(),
    ]

try:
    pdf_ROC= PdfPages(os.path.join(output_folder,f"ROC of {hour} hour results_regression.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"ROC of {hour} hour results_regression.pdf"))
    pdf_ROC= PdfPages(os.path.join(output_folder,f"ROC of {hour} hour results_regression.pdf"))

for data_name, xd in zip(names, x_data):
    columns = xd.columns.tolist()
    x_train_d = x_train[columns]
    x_test_d = x_test[columns]

    if 'Wrap' in data_name:
        wrapper += 1
    else:
        wrapper=0

    temp_fpr=dict()
    temp_tpr=dict()
    temp_auc=dict()

    for name, clf_s in zip(names, classifiers):

        clf,y_pred_clas,y_pred_prob,y_test,x_test_d = machine_learning_function(x_train_d,x_test_d,y_train,y_test,clf_s,wrapper)


        fpr, tpr, _ = roc_curve(y_test,  y_pred_prob)
        auc = roc_auc_score(y_test, y_pred_prob)


        temp_fpr.update({f'{name}': fpr})
        temp_tpr.update({f'{name}': tpr})
        temp_auc.update({f'{name}': auc})
    a=0
    for k,v in temp_fpr.items():
        plt.plot(v,temp_tpr.get(k),colours[a],label=f"{k}, auc={temp_auc.get(k)}",linewidth=1.5,markersize=1)
        a= a+1
    plt.plot(fpr_pim,tpr_pim,'b->',label=f"PIM3Score, auc={auc_pim}",linewidth=1.5,markersize=1)
    plt.legend(loc=4,fontsize='xx-small')
    plt.title(f'{data_name} ROC of {hour} hour data')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    axes = plt.gca()
    axes.set_xlim([0,1])
    axes.set_ylim([0,1])
    fig=plt.gcf()
    pdf_ROC.savefig(fig)
    plt.close(fig)

    del temp_fpr,temp_tpr,temp_auc
pdf_ROC.close()


        
#score metrics for regression classifiers
"""""
print(metrics.explained_variance_score(y_test,y_pred))
print(metrics.max_error(y_test,y_pred))
print(metrics.mean_absolute_error(y_test,y_pred))
print(metrics.mean_squared_error(y_test,y_pred))
print(metrics.mean_squared_error(y_test,y_pred))
print(metrics.median_absolute_error(y_test,y_pred))
print(metrics.r2_score(y_test,y_pred))
"""

In [ ]:
print("Original ROC area: {:0.3f}".format(roc_auc_score(y_true, y_pred)))

n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred), len(y_pred))
    if len(np.unique(y_true[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(y_true[indices], y_pred[indices])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))


from sklearn.metrics import roc_auc_score
from math import sqrt

def roc_auc_ci(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2*AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC*(1 - AUC) + (N1 - 1)*(Q1 - AUC**2) + (N2 - 1)*(Q2 - AUC**2)) / (N1*N2))
    lower = AUC - 1.96*SE_AUC
    upper = AUC + 1.96*SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower, upper)
a=roc_auc_ci(y_true,y_pred)
print(a)

In [ ]:
"""
This is used to calculate the absolute feature importances of a classifier, with the current code, this snippet does not currently work
"""

"""
for k,v in models.items():
    clf,y_pred,y_test,float_colums,cat_list = machine_learning_function(x_sp,y,v,k)
    #onehot_columns = list(clf.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names(input_features=cat_list))
    #float_colums.extend(onehot_columns)
    try:
        coefs = clf.named_steps["classifier"].coef_.flatten()
    except:
        coefs=clf.named_steps["classifier"].feature_importances_
    import pandas as pd
    # Zip coefficients and names together and make a DataFrame
    zipped = zip(float_colums, coefs)
    df = pd.DataFrame(zipped, columns=["feature", "value"])
    # Sort the features by the absolute value of their coefficient
    df["abs_value"] = df["value"].apply(lambda x: abs(x))
    df["colors"] = df["value"].apply(lambda x: "green" if x > 0 else "red")
    df = df.sort_values("abs_value", ascending=False)


    import seaborn as sns
    fig, ax = plt.subplots(1, 1, figsize=(15, 4))
    sns.barplot(x="feature",
                y="value",
                data=df.head(30),
                palette=df.head(30)["colors"])
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=12)
    ax.set_title("Top 20 Features", fontsize=12)
    ax.set_ylabel("Coef", fontsize=12)
    ax.set_xlabel("Feature Name", fontsize=12)
    print(df["feature"].head(20).tolist())
"""

In [ ]:
"""
from sklearn import cluster as clus
from sklearn import metrics
from scipy.cluster.hierarchy import dendrogram
float_colums=list(x.select_dtypes(include=['float64']).columns)
int_columns=list(x.select_dtypes(include=['int8']).columns)
cat_list=list((Counter(list(x.columns))-Counter(float_colums)-Counter(int_columns)).elements())
float_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent'))])
int_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='constant',fill_value=0))])

cat_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='constant',fill_value='Unknown')),('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocess = ColumnTransformer(transformers=[('float',float_transformer,float_colums),('int',int_transformer,int_columns),('cat',cat_transformer,cat_list)],                 remainder='passthrough')
x_fit=preprocess.fit_transform(x)
onehot_columns = list(preprocess.named_transformers_['cat'].named_steps['onehot'].get_feature_names(input_features=cat_list))
int_columns.extend(float_columns)
int_columns.extend(onehot_columns)




labels_true=y
af = clus.AffinityPropagation(max_iter=500,preference=-1).fit(x_fit)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_

n_clusters_ = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(x_fit, labels, metric='sqeuclidean'))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt
from itertools import cycle

plt.close('all')
plt.figure(1)
plt.clf()

colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    class_members = labels == k
    cluster_center = x_fit[cluster_centers_indices[k]]
    plt.plot(x_fit[class_members,0],x_fit[class_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
    for xs in x_fit[class_members]:
        plt.plot([cluster_center[0], xs[0]], [cluster_center[1], xs[1]], col)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()
"""